## Background

This notebook provides a simplified way to interact with selected Maxar Open Data STAC collections. 

It assumes you've run the repo installation instructions (`make init`) and have the Docker services running.

## 1. Imports and Setup

We'll import the necessary libraries: `APIClient` to discover data and `MaxarCollection` to interact with a specific collection.

In [1]:
from datetime import UTC, datetime

from eo_maxar.client import APIClient
from eo_maxar.collection import MaxarCollection

## 2. Discover Available Collections

First, we instantiate the `APIClient` and use it to see which data collections are available in our local `pgSTAC` database.

In [2]:
client = APIClient()
available_collections = client.get_all_collections()
available_collections

['MAXAR_afghanistan_earthquake22',
 'MAXAR_BayofBengal_Cyclone_Mocha_May_23',
 'MAXAR_cyclone_emnati22',
 'MAXAR_Emilia_Romagna_Italy_flooding_may23',
 'MAXAR_Gambia_flooding_8_11_2022',
 'MAXAR_ghana_explosion22',
 'MAXAR_Hurricane_Fiona_9_19_2022',
 'MAXAR_Hurricane_Ian_9_26_2022',
 'MAXAR_Hurricane_Idalia_Florida_Aug23',
 'MAXAR_India_Floods_Oct_2023',
 'MAXAR_Indonesia_Earthquake22',
 'MAXAR_Kahramanmaras_turkey_earthquake_23',
 'MAXAR_Kalehe_DRC_Flooding_5_8_23',
 'MAXAR_kentucky_flooding_7_29_2022',
 'MAXAR_Libya_Floods_Sept_2023',
 'MAXAR_Marshall_Fire_21_Update',
 'MAXAR_Maui_Hawaii_fires_Aug_23',
 'MAXAR_McDougallCreekWildfire_BC_Canada_Aug_23',
 'MAXAR_Morocco_Earthquake_Sept_2023',
 'MAXAR_Nepal_Earthquake_Nov_2023',
 'MAXAR_New_Zealand_Flooding23',
 'MAXAR_NWT_Canada_Aug_23',
 'MAXAR_pakistan_flooding22',
 'MAXAR_shovi_georgia_landslide_8Aug23',
 'MAXAR_southafrica_flooding22',
 'MAXAR_Sudan_flooding_8_22_2022',
 'MAXAR_tonga_volcano21',
 'MAXAR_volcano_indonesia21',
 'MAXA

## 3. Analyze a Collection: 2023 Turkey & Syria Earthquake

We'll start by analyzing the imagery from the earthquake in Turkey and Syria.

In [4]:
collection_id = "MAXAR_Kahramanmaras_turkey_earthquake_23"

turkey_earthquake = MaxarCollection(collection_id=collection_id)

event_date = datetime(2023, 2, 6, tzinfo=UTC)

### 3.1. Get Collection Information

We can easily access the collection's metadata through the `.info` property. It returns a validated `pydantic` model.

In [5]:
for field, value in turkey_earthquake.info.model_dump(exclude_none=True).items():
    print(f"{field}: {value}")

id: MAXAR_Kahramanmaras_turkey_earthquake_23
title: Turkey and Syria Earthquake 2023
description: Maxar OpenData | A devastating magnitude 7.8 earthquake struck the Turkish province of Kahramanmaras, approximately 23 kilometers east of Nurdagi in the Gaziantep province near the Syrian border, at 4:17 a.m. local time on Monday, February 6, 2023, followed by a 7.5 magnitude aftershock nine hours later. More than 6,000 people have died in Turkey and Syria, and tens of thousands of people have been injured. Those numbers are expected to increase as search and rescue activities continue. At least 13 million people in the region have been impacted by the earthquake and aftershock. Turkey's president declared a three-month state of emergency in the 10 provinces hardest hit by the earthquake.
extent: {'spatial': {'bbox': [[35.302597, 35.875122, 40.310497, 38.47292570695286], [37.2976, 36.98959965805714, 37.47444448907068, 37.015901889979396], [37.29774464331677, 36.9896650792383, 37.4575240343

In [6]:
turkey_earthquake.collection_bbox_map()

Map(center=[37.17402385347643, 37.806546999999995], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### 3.3. Map Pre- and Post-Event Item Availability

We can create a map that colours each item's footprint based on whether it was captured before or after the earthquake.

- **Blue**: Pre-event imagery
- **Red**: Post-event imagery

In [7]:
turkey_earthquake.pre_post_map(event_date=event_date)

Map(center=[36.10519545142351, 36.404020998028585], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### 3.4. View a Single Image (COG)

The raster API (`titiler-pgstac`) can serve tiles directly from a single Cloud-Optimized GeoTIFF (COG). Let's view the `visual` asset of the first item in the collection.

One thing to note - getting the collections info or items will automatically cache them behind the scenes using a `cached_property` decorator, which means we don't need to spend time repeating calls to the docker services.

In [8]:
first_item_id = turkey_earthquake.items[0].id

turkey_earthquake.single_cog_map(item_id=first_item_id, map_kwargs={"zoom": 14})

Map(center=[36.10519545142351, 36.404020998028585], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### 3.5. Create Dynamic Mosaics

Probably the most feature is creating virtual mosaics on-the-fly. We'll define a bounding box around Antakya, Turkey, to see the impact there.

#### Pre-Earthquake Mosaic

In [9]:
bounds = [36.129570, 36.180701, 36.194458, 36.224058]

map_kwargs = {
    "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
    "zoom": 16,
    "layout": {"height": "600px"},
}

turkey_earthquake.pre_event_mosaic_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs=map_kwargs,
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

#### Post-Earthquake Mosaic

And for post-eathquake.

In [10]:
turkey_earthquake.post_event_mosaic_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs=map_kwargs,
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

### 3.6. Create a Split-View Comparison Map

Finally, we can combine the pre- and post-event mosaics into a single interactive split-view map for easy comparison.

In [11]:
turkey_earthquake.mosaic_split_map(
    bbox=bounds,
    event_date=event_date,
    map_kwargs=map_kwargs,
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…